# Diffusion Image Alignment

This Notebook takes two Diffusion weighted images as an input and alignes the second one to the first one. Additionally it also rotates the bvecs of the second image according to the performed rotation of the image itself.

In [43]:
# Transforms Image 2 to Image 1, saves the reorientated Images 2
path1 = "/scratch/cpoetter/local_regression/"
nifti_name1 = "dwi_ec_motion_corrected.nii.gz"
bvals_name1 = "dwi_ec_motion_corrected_alligned.bval"
bvecs_name1 = "dwi_ec_motion_corrected.bvec"

path2 = "/scratch/cpoetter/100bzeros/11782/"
nifti_name2 = "0011_01_dwi_mux3_pe0_allb0.nii.gz"
bvals_name2 = "0011_01_dwi_mux3_pe0_allb0.bval"
bvecs_name2 = "0011_01_dwi_mux3_pe0_allb0.bvec"

path_saving = "/scratch/cpoetter/local_regression/"
nifti_name2_new = "0011_01_dwi_mux3_pe0_allb0_reorientated.nii.gz"
bvecs_name2_new = "0011_01_dwi_mux3_pe0_allb0_reorientated.bvec"

In [44]:
import nibabel as nib
import nipy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dipy.viz import regtools
import nipy.algorithms.registration
import ip_utils
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table
import multiprocessing
import tempfile
import os
from utilities import *
from dipy.align.imaffine import AffineMap
from parallelization import *

In [45]:
# Loading nifti image, data, affine and gtabs
def load_nifti_image(path, nifti_name, bvals_name, bvecs_name):
    nifti_path = path + nifti_name
    bvals_path = path + bvals_name
    bvecs_path = path + bvecs_name

    img = nib.load(nifti_path)
    data = img.get_data()
    affine = img.affine
    header = img.header
    
    bvals, bvecs = read_bvals_bvecs(bvals_path, bvecs_path)
    gtab = gradient_table(bvals, bvecs)
    
    print 'Image %s loaded' % nifti_name
    return img, data, affine, header, gtab

In [46]:
def save_bvecs(bvecs, path, name):
    with open(path + name, 'w') as fp:
        fp.write(' '.join(['%0.4f' % bv for bv in np.squeeze(np.asarray(bvecs[:, 0]))]) + '\n')
        fp.write(' '.join(['%0.4f' % bv for bv in np.squeeze(np.asarray(bvecs[:, 1]))]) + '\n')
        fp.write(' '.join(['%0.4f' % bv for bv in np.squeeze(np.asarray(bvecs[:, 2]))]) + '\n')
    print 'bvecs %s successfully saved' % name

In [47]:
# Rotation of bvecs
def reorient_bvecs(transforms, bvecs):
    r = transforms.rotation
    R = np.matrix([[1.,0,0], [0,np.cos(r[0]),np.sin(r[0])], [0,-np.sin(r[0]),np.cos(r[0])]])
    R = R*np.matrix([[np.cos(r[1]),0,np.sin(r[1])], [0,1,0], [-np.sin(r[1]),0,np.cos(r[1])]])
    R = R*np.matrix([[np.cos(r[2]),np.sin(r[2]),0], [-np.sin(r[2]),np.cos(r[2]),0], [0,0,1]])
    R = np.linalg.inv(R)
    bvecs = (R.T * np.matrix(bvecs).T).T
    print "bvecs rotated"
    return bvecs

In [48]:
def average_image(data, gtab, affine, header):
    # If image is already 3 dimentional do not calculate the average
    squeezed_data = data.squeeze()
    if squeezed_data.ndim == 4:
        data_average = np.mean(squeezed_data[...,gtab.b0s_mask], axis=3)       
    else:
        data_average = squeezed_data
    
    img_average = nib.Nifti1Image(data_average.astype(float), affine, header=header) 
    print "Average image calculated"
    return img_average

In [49]:
def reorientate_image(img1, data1, gtab1, img2, data2, gtab2):
    img1_average = average_image(data1, gtab1, img1.affine, img1.header)     
    img2_average = average_image(data2, gtab2, img2.affine, img2.header)
       
    reg = nipy.algorithms.registration.HistogramRegistration(img1_average, img2_average, similarity='crl1', interp='tri')
    T = reg.optimize('rigid')
    
    # check if 3D or 4D image
    if data2.squeeze().ndim == 4:
        img2_splitted = nib.four_to_three(img2)
        n = data2.squeeze().shape[-1]
    else: 
        img2_splitted = [img2]
        n = 1
    
    p = parallelization(display=False)
    image_list_rotated  = p.start(nipy.algorithms.registration.resample, n, img2_splitted, [T], reference=[img1_average], interp_order=[1])
    
    # resample loses header, thereofre add old header
    for i in range(len(image_list_rotated)):
        image_list_rotated[i] = nib.Nifti1Image(image_list_rotated[i].get_data(), image_list_rotated[i].affine, header=img2_splitted[i].header) 
    
    img2_new = nib.funcs.concat_images(image_list_rotated)
    
    bvecs_rotated = reorient_bvecs(T, gtab2.bvecs)
    
    print "Image 2 successfully reorientated"
    return img2_new, bvecs_rotated

In [50]:
img1, data1, affine1, header1, gtab1 = load_nifti_image(path1, nifti_name1, bvals_name1, bvecs_name1)
img2, data2, affine2, header2, gtab2 = load_nifti_image(path2, nifti_name2, bvals_name2, bvecs_name2)

Image dwi_ec_motion_corrected.nii.gz loaded
Image 0011_01_dwi_mux3_pe0_allb0.nii.gz loaded


In [51]:
img2_new, bvecs_rotated = reorientate_image(img1, data1, gtab1, img2, data2, gtab2)

Average image calculated
Average image calculated
Initial guess...
translation : [ 0.  0.  0.]
rotation    : [ 0.  0.  0.]

Optimizing using fmin_powell
translation : [-1.618034  1.        1.      ]
rotation    : [ 0.00381966  0.01       -0.04236068]

crl1 = 0.594699468226

translation : [-1.5483048  -1.26802381  1.381966  ]
rotation    : [ 0.04821611  0.03618034 -0.07361105]

crl1 = 0.639812776412

translation : [-1.7447585  -0.26802381  1.763932  ]
rotation    : [ 0.05821611  0.04618034 -0.06361105]

crl1 = 0.655306822785

translation : [-2.46649332  0.25984021  1.80637842]
rotation    : [ 0.05731441  0.05527864 -0.05834478]

crl1 = 0.660641884974

Optimization terminated successfully.
         Current function value: -0.660642
         Iterations: 4
         Function evaluations: 155
bvecs rotated
Image 2 successfully reorientated


In [52]:
save_bvecs(bvecs_rotated, path_saving, bvecs_name2_new)

image2_save = nib.Nifti1Image(img2_new.get_data(), img2_new.affine)
nib.save(image2_save,  path_saving + nifti_name2_new)
print "Reorientated image %s saved" % nifti_name2_new

bvecs 0011_01_dwi_mux3_pe0_allb0_reorientated.bvec successfully saved
Reorientated image 0011_01_dwi_mux3_pe0_allb0_reorientated.nii.gz saved
